# Checkpoint 3 - Comparações dados do Brasil

## Carregar módulos

In [ ]:
import numpy as np
import pandas as pd

## Carregar dados

- Dados Covid Brasil

In [ ]:
! wget --no-check-certificate --content-disposition 'https://github.com/wcota/covid19br/blob/master/cases-brazil-cities-time.csv.gz?raw=true'

--2025-05-15 23:09:17--  https://github.com/wcota/covid19br/blob/master/cases-brazil-cities-time.csv.gz?raw=true
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/wcota/covid19br/raw/refs/heads/master/cases-brazil-cities-time.csv.gz [following]
--2025-05-15 23:09:17--  https://github.com/wcota/covid19br/raw/refs/heads/master/cases-brazil-cities-time.csv.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wcota/covid19br/refs/heads/master/cases-brazil-cities-time.csv.gz [following]
--2025-05-15 23:09:17--  https://raw.githubusercontent.com/wcota/covid19br/refs/heads/master/cases-brazil-cities-time.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubus

In [ ]:
! gunzip cases-brazil-cities-time.csv.gz

In [ ]:
dados_brasil = pd.read_csv('cases-brazil-cities-time.csv')

- Dados censo

In [ ]:
dados_censo = pd.read_csv('dados_municipios_2010.csv', sep = ';', decimal = ',', encoding = 'latin1')

## Ajustar dados

In [ ]:
max_date = max(dados_brasil['date'])

In [ ]:
dados_brasil = dados_brasil[(dados_brasil['date'] == max_date) & \
                            (dados_brasil['state'] != 'TOTAL')]

In [ ]:
cols = ['ibgeID', 'date', 'state', 'city', 'totalCases', 'deaths']

In [ ]:
dados_brasil = dados_brasil[cols].reset_index(drop=True)

## Aqui começa o trabalho

In [ ]:
dados_brasil

,ibgeID,date,state,city,totalCases,deaths
0,5200050,2023-03-18,GO,Abadia de Goiás/GO,4074,50
1,3100104,2023-03-18,MG,Abadia dos Dourados/MG,1943,21
2,5200100,2023-03-18,GO,Abadiânia/GO,2218,55
3,1500107,2023-03-18,PA,Abaetetuba/PA,12009,244
4,3100203,2023-03-18,MG,Abaeté/MG,4279,51
...,...,...,...,...,...,...
5591,4301552,2023-03-18,RS,Áurea/RS,665,7
5592,4101150,2023-03-18,PR,Ângulo/PR,967,11
5593,2900504,2023-03-18,BA,Érico Cardoso/BA,2467,53
5594,1505106,2023-03-18,PA,Óbidos/PA,8717,166


In [ ]:
dados_censo

,ibgeID,Municipio,ESPVIDA,E_ANOSESTUDO,T_ANALF18M,RDPC,IDHM
0,1100015,ALTA FLORESTA D'OESTE,70.75,8.67,13.00,476.99,0.641
1,1100023,ARIQUEMES,73.36,9.18,8.53,689.95,0.702
2,1100031,CABIXI,70.39,10.09,14.73,457.17,0.650
3,1100049,CACOAL,74.27,9.74,8.94,738.06,0.718
4,1100056,CEREJEIRAS,72.94,10.22,11.17,577.18,0.692
...,...,...,...,...,...,...,...
5560,5222005,VIANÓPOLIS,74.10,10.11,8.52,664.33,0.712
5561,5222054,VICENTINÓPOLIS,73.25,10.07,13.51,651.07,0.684
5562,5222203,VILA BOA,74.07,10.49,17.07,376.72,0.647
5563,5222302,VILA PROPÍCIO,73.09,9.46,20.75,401.84,0.634


In [ ]:
dados_brasil['ibgeID'] = dados_brasil['ibgeID'].astype(str)
dados_censo['ibgeID'] = dados_censo['ibgeID'].astype(str)

df = pd.merge(dados_brasil, dados_censo, on='ibgeID', how='left')

df = df.dropna()

print("Tamanho do dataset após o join:", df.shape)


Tamanho do dataset após o join: (5565, 12)


In [ ]:

sudeste = df[df['state'].isin(['SP', 'RJ', 'MG', 'ES'])]

variancia = sudeste['RDPC'].var()


z = 1.96
e = 20

n_amostra = int((z**2 * variancia) / (e**2))
print("Tamanho da amostra:", n_amostra)

amostra_sudeste = sudeste.sample(n=n_amostra, random_state=42)


Tamanho da amostra: 443


In [ ]:
from scipy.stats import ttest_ind

def teste_t(grupo1, grupo2, coluna):
    g1 = amostra_sudeste[amostra_sudeste['state'] == grupo1][coluna]
    g2 = amostra_sudeste[amostra_sudeste['state'] == grupo2][coluna]
    stat, p = ttest_ind(g1, g2, equal_var=False)

    print(f"\n{grupo1} vs {grupo2} | {coluna} | p-valor = {p:.4f}")
    if p < 0.05:
        print("→ Rejeitamos H0: Diferença significativa")
    else:
        print("→ Não rejeitamos H0: Sem diferença significativa")

# a) MG vs SP – totalCases
teste_t('MG', 'SP', 'totalCases')

# b) RJ vs MG – deaths
teste_t('RJ', 'MG', 'deaths')

# c) ES vs RJ – RDPC
teste_t('ES', 'RJ', 'RDPC')

# d) SP vs RJ – T_ANALF18M
teste_t('SP', 'RJ', 'T_ANALF18M')



MG vs SP | totalCases | p-valor = 0.2634
→ Não rejeitamos H0: Sem diferença significativa

RJ vs MG | deaths | p-valor = 0.2739
→ Não rejeitamos H0: Sem diferença significativa

ES vs RJ | RDPC | p-valor = 0.0112
→ Rejeitamos H0: Diferença significativa

SP vs RJ | T_ANALF18M | p-valor = 0.6410
→ Não rejeitamos H0: Sem diferença significativa


In [ ]:
from scipy.stats import chi2_contingency

cols_categorias = ['totalCases', 'deaths', 'T_ANALF18M', 'RDPC']
cols_names = [c + '_cat' for c in cols_categorias]

amostra_sudeste[cols_names] = amostra_sudeste.groupby('state')[cols_categorias] \
    .transform(lambda x: pd.qcut(x, 2, labels=['baixo', 'alto']))


def teste_quiquadrado(var1, var2):
    tabela = pd.crosstab(amostra_sudeste[var1], amostra_sudeste[var2])
    chi2, p, dof, _ = chi2_contingency(tabela)
    print(f"\nAssociação entre {var1} e {var2} | p-valor = {p:.4f}")
    if p < 0.05:
        print("→ Rejeitamos H0: Existe associação significativa")
    else:
        print("→ Não rejeitamos H0: Não há associação significativa")

# a) totalCases_cat x deaths_cat
teste_quiquadrado('totalCases_cat', 'deaths_cat')

# b) RDPC_cat x T_ANALF18M_cat
teste_quiquadrado('RDPC_cat', 'T_ANALF18M_cat')



Associação entre totalCases_cat e deaths_cat | p-valor = 0.0000
→ Rejeitamos H0: Existe associação significativa

Associação entre RDPC_cat e T_ANALF18M_cat | p-valor = 0.0000
→ Rejeitamos H0: Existe associação significativa
